In [1]:
import gzip
import json
import math
from collections import defaultdict
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Flatten, Concatenate, Dense, Multiply
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [2]:
path = 'goodreads_interactions_poetry.json.gz'

dataset = []

with gzip.open(path, 'rt', encoding='utf8') as file:
    # Process the file line by line
    for line in file:
        # Decode each line from JSON format
        json_content = json.loads(line)
        dataset.append(json_content)


In [3]:
usersPerItem = defaultdict(set)
itemsPerUser = defaultdict(set)
reviewsPerUser = defaultdict(list)
reviewsPerItem = defaultdict(list)
ratingsPerUser = defaultdict(list)

In [5]:
for d in dataset:
    user,item, rating = d['user_id'], d['book_id'], d['rating']
    usersPerItem[item].add(user)
    itemsPerUser[user].add(item)
    reviewsPerUser[user].append(d)
    ratingsPerUser[user].append(rating)
    reviewsPerItem[item].append(d)

In [7]:
uniqueItems = usersPerItem.keys()
uniqueUsers = itemsPerUser.keys()

In [10]:
ncf_model_with_gmf = tf.keras.models.load_model('ncf_with_gmf_model.h5')
